In [1]:
# Imports
import os
import re
import time
import json
import csv
import random
import spacy
import pandas as pd

In [ ]:
dir = os.getcwd()
data_dir = os.path.join(dir, 'data')
os.makedirs(data_dir, exist_ok=True)
output_dir = os.path.join(dir, 'output')
os.makedirs(output_dir, exist_ok=True)

: 

In [3]:
with open(os.path.join(data_dir, f"gpt4_answers.jsonl"), 'r', encoding='utf-8') as jsonl_file:
    gpt4_answers = [json.loads(line) for line in jsonl_file]

In [4]:
with open(os.path.join(data_dir, f"llama_answers.jsonl"), 'r', encoding='utf-8') as jsonl_file:
    llama_answers = [json.loads(line) for line in jsonl_file]

In [5]:
with open(os.path.join(data_dir, f"physician_answers.jsonl"), 'r', encoding='utf-8') as jsonl_file:
    physician_answers = [json.loads(line) for line in jsonl_file]

# Create Coarse Part 1 and Part 2

In [7]:
# Randomizing which author is in which group:
# the first three authors do annotations 0-50 with coarse style
# the second three authors do annotations 51-100 with fine style
random.seed(17)

annotators = [f"Annotator{i+1}" for i in range(6)]
random.shuffle(annotators)
annotators

['Annotator1',
 'Annotator6',
 'Annotator2',
 'Annotator3',
 'Annotator4',
 'Annotator5']

In [8]:
indexes_dict = {'group0' : {'coarse': [0, 50],
                            'fine': [50, 100]},
                'group1' : {'coarse': [50, 100],
                            'fine': [0, 50]}}

In [ ]:
flag = 0
for annotator in annotators:
    
    print(annotator)
    
    if flag < 3:
        indexes_dictionary = indexes_dict['group0']
    else:
        indexes_dictionary = indexes_dict['group1']
    
    print(indexes_dictionary)
        
    for annotation_type, indexes in indexes_dictionary.items():
        
        print(f'Adding {annotation_type} annotations, indexes: {indexes}')
        
        question_positions = list(range(indexes[0], indexes[1])) # positions of the questions in the jsonl files that store questions and answers
        random.shuffle(question_positions) # Randomize question order but randomizing order of indexes 

        # Split questions into batches of 3
        batches = {}
        batch_n = 1
        for i in range(0, len(question_positions), 3):
            key = f'batch_{batch_n}'
            batches[key] = question_positions[i:i+3]
            batch_n += 1
        
        for batch_id, positions in batches.items():
            
            for n in positions:
                
                qa_pairs = [gpt4_answers[n], llama_answers[n], physician_answers[n]] # Randomizing answer type order
                random.shuffle(qa_pairs)
                
                if qa_pairs[0]['question_id'] == qa_pairs[1]['question_id'] == qa_pairs[2]['question_id']:
                
                    for qa_pair in qa_pairs:
                        
                        if annotation_type == 'coarse':
                            qa_pair['annotation_type'] = annotation_type
                            qa_pair['rated'] = 'No'
                            qa_pair['batch_id'] = batch_id
                        
                            # with open(os.path.join(output_dir, 'all', f'{annotator.lower()}_{annotation_type}.jsonl'), 'a') as file:
                            #         json.dump(qa_pair, file)
                            #         file.write('\n')
                    
                        # elif annotation_type == 'fine':
                            
                        #     sentences = bold_sentences(qa_pair['answer'])
                            
                        #     for sentence in sentences:
                        #         new_d = qa_pair.copy()
                        #         new_d['annotation_type'] = annotation_type
                        #         new_d['rated'] = 'No'
                        #         new_d['batch_id'] = batch_id
                        #         new_d['sentence_id'] = new_d['answer_id'] + f'_{sentence[0]}'
                        #         new_d['answer'] = sentence[1]
                        #         with open(os.path.join(output_dir, 'all', f'{annotator.lower()}_{annotation_type}.jsonl'), 'a') as file:
                        #             json.dump(new_d, file)
                        #             file.write('\n')

    flag += 1

Annotator1
{'coarse': [0, 50], 'fine': [50, 100]}
Adding coarse annotations, indexes: [0, 50]
Adding fine annotations, indexes: [50, 100]
Annotator6
{'coarse': [0, 50], 'fine': [50, 100]}
Adding coarse annotations, indexes: [0, 50]
Adding fine annotations, indexes: [50, 100]
Annotator2
{'coarse': [0, 50], 'fine': [50, 100]}
Adding coarse annotations, indexes: [0, 50]
Adding fine annotations, indexes: [50, 100]
Annotator3
{'coarse': [50, 100], 'fine': [0, 50]}
Adding coarse annotations, indexes: [50, 100]
Adding fine annotations, indexes: [0, 50]
Annotator4
{'coarse': [50, 100], 'fine': [0, 50]}
Adding coarse annotations, indexes: [50, 100]
Adding fine annotations, indexes: [0, 50]
Annotator5
{'coarse': [50, 100], 'fine': [0, 50]}
Adding coarse annotations, indexes: [50, 100]
Adding fine annotations, indexes: [0, 50]


# Create Fine Part 1 and 2

## Dealing with numbered lists

In [ ]:
wrong = []
for l in [gpt4_answers, llama_answers]:
    for d in l:
        if "\n\n1." in d['answer'] or '\n\n' in d['answer'] or "\n*" in d['answer'] or '**' in d['answer']:
            wrong.append(d['answer_id'])
set(wrong)

{'gpt4_0',
 'gpt4_10',
 'gpt4_13',
 'gpt4_14',
 'gpt4_15',
 'gpt4_17',
 'gpt4_18',
 'gpt4_19',
 'gpt4_2',
 'gpt4_3',
 'gpt4_30',
 'gpt4_35',
 'gpt4_36',
 'gpt4_47',
 'gpt4_56',
 'gpt4_68',
 'gpt4_72',
 'gpt4_73',
 'gpt4_81',
 'gpt4_87',
 'gpt4_88',
 'gpt4_92',
 'llama_0',
 'llama_1',
 'llama_10',
 'llama_11',
 'llama_13',
 'llama_14',
 'llama_15',
 'llama_17',
 'llama_18',
 'llama_19',
 'llama_20',
 'llama_24',
 'llama_27',
 'llama_30',
 'llama_31',
 'llama_32',
 'llama_34',
 'llama_35',
 'llama_36',
 'llama_4',
 'llama_43',
 'llama_44',
 'llama_47',
 'llama_49',
 'llama_5',
 'llama_51',
 'llama_52',
 'llama_53',
 'llama_54',
 'llama_55',
 'llama_56',
 'llama_57',
 'llama_59',
 'llama_6',
 'llama_61',
 'llama_63',
 'llama_64',
 'llama_65',
 'llama_66',
 'llama_67',
 'llama_68',
 'llama_69',
 'llama_7',
 'llama_70',
 'llama_71',
 'llama_72',
 'llama_73',
 'llama_74',
 'llama_76',
 'llama_78',
 'llama_79',
 'llama_84',
 'llama_85',
 'llama_86',
 'llama_87',
 'llama_88',
 'llama_89',
 'll

In [ ]:
len(set(wrong))

86

In [ ]:
# Process the text with spaCy to segment into sentences
wrong_ones = []
for n in [1,3]:
    
    with open(os.path.join(output_dir, 'all',f'annotator{n}_coarse.jsonl'), 'r', encoding='utf-8') as jsonl_file:
        data = [json.loads(line) for line in jsonl_file]
        
        for d in data:
            if d['answer_id'] in set(wrong):
                new_d = {'answer_id': d['answer_id'],
                        'answer': d['answer']}
                wrong_ones.append(new_d)

In [ ]:
for d in wrong_ones:
    text = d['answer']
    text = text.replace(". ", ". #")
    text = text.replace("\n- ", "#\n- #")
    text = text.replace("\n* ", "#\n* #")
    text = text.replace("\n\n", "#\n\n#")
    d['answer'] = text

In [ ]:
def special_bolding(text):

    sentences = text.split("#")
    
    fine_sentences = []
    for bold_index in range(len(sentences)):
        if len(sentences[bold_index]) > 5:
            bold_sentence = f':orange[{sentences[bold_index]}]'
            new_sentences = sentences[:bold_index] + [bold_sentence] + sentences[bold_index + 1:]
            # print('new sentences!')
            # print(new_sentences)
            fine_sentence = ' '.join(new_sentences)
            clean_sentence = re.sub(r"\n([2-7]\. )\]", r"]\n\1", fine_sentence)
            fine_sentences.append((len(fine_sentences), clean_sentence))

    return fine_sentences

# Example text
text = """This is the first sentence. #\n\n1.#And another sentence.#\n\n#Here's the second one! #Is this the third sentence? Yes, it is."""

# bold sentences in the example text
for i in special_bolding(text):
    print(i)

(0, ":orange[This is the first sentence. ] \n\n1. And another sentence. \n\n Here's the second one!  Is this the third sentence? Yes, it is.")
(1, "This is the first sentence.  \n\n1. :orange[And another sentence.] \n\n Here's the second one!  Is this the third sentence? Yes, it is.")
(2, "This is the first sentence.  \n\n1. And another sentence. \n\n :orange[Here's the second one! ] Is this the third sentence? Yes, it is.")
(3, "This is the first sentence.  \n\n1. And another sentence. \n\n Here's the second one!  :orange[Is this the third sentence? Yes, it is.]")


In [ ]:
for d in wrong_ones:
    for version in special_bolding(d['answer']):
        new_d = {'text': version[1]}
        # with open(os.path.join(f'test.jsonl'), 'a') as file:
        #     json.dump(new_d, file)
        #     file.write('\n')   

In [ ]:
wrong_ones_dict = {}
for d in wrong_ones:
    answer_id = d['answer_id']
    wrong_ones_dict[answer_id] = special_bolding(d['answer'])
    
# with open(os.path.join('data', f'wrong_ones.json'), 'w') as file:
#     json.dump(wrong_ones_dict, file, indent=4)

"A white tongue can be caused by several conditions, some of which are benign and easily treatable, while others may require more comprehensive medical intervention. #Common causes include:#\n\n#1. #Oral hygiene: Poor oral hygiene can lead to a buildup of bacteria and debris on the tongue, presenting as a white coating.\n2. #Dehydration or dry mouth: This can cause the surface of the tongue to appear white due to decreased saliva production.\n3. #Oral thrush: This is a fungal infection caused by Candida yeast, leading to white, creamy patches on the tongue and inside the mouth.\n4. #Leukoplakia: This condition involves thick, white patches on the tongue and is often associated with smoking or other irritants. #It can sometimes be a precancerous condition.\n5. #Smoking and alcohol use: Both can contribute to changes in the color and texture of the tongue.#\n\n#If the white tongue persists, it's advisable to consult a healthcare provider to diagnose the underlying cause and receive appropriate treatment."

In [ ]:
# wrong_ones_df.to_csv("wrong_ones.csv", index=False)

In [ ]:
# with open(os.path.join('data', 'wrong_ones.json'), 'r') as json_file:
#     wrong_ones = json.load(json_file)

# new_wrong_ones = {}
# for answer_id, sentences in wrong_ones.items():
#     new_sentences = []
#     for number, sentence in sentences:
#         new_sentence = sentence.replace(":orange[", "<mark>").replace("]", "</mark>")
#         new_sentences.append([number, new_sentence])
#     new_wrong_ones[answer_id] = new_sentences

# with open(os.path.join('data', f'new_wrong_ones.json'), 'w') as file:
#     json.dump(new_wrong_ones, file, indent=4)

## Functions to split and highlight sentences

In [6]:
def merge_short_sentences(sentences, min_length=30):
    merged = []
    i = 0

    while i < len(sentences):
        current = sentences[i]
        if len(current) < min_length and '\n\n' not in current and i + 1 < len(sentences):
            # Merge with the next sentence
            merged_sentence = current.strip() + " " + sentences[i + 1].strip()
            merged.append(merged_sentence)
            i += 2  # Skip the next one since it's been merged
        else:
            merged.append(current)
            i += 1

    return merged


In [7]:
def clean_fine_sentence(text):

    # Replace "\n\n]" with "]\n\n"
    text = text.replace("\n\n]", "]\n\n")
    # Replace "\n]" with "]\n"
    text = text.replace("\n]", "]\n")
    # Replace "\n\nX.]" with "]\n\nX." where X is 1–7
    text = re.sub(r'\n\n([1-7])\.\]', r']\n\n\1.', text)
    # Replace ":orange[X." with "X. :orange[" where X is a number between 1 and 7
    re.sub(r':orange\[(?P<num>[1-7])\. (.*?)\]', r'\g<num>. :orange[\2]', text)
    
    return text

In [8]:
def bold_sentences(text):
    
    # Load the spaCy English model
    nlp = spacy.load("en_core_web_sm")
    # Process the text with spaCy to segment into sentences
    doc = nlp(text)
    split_sentences = [sentence.text for sentence in doc.sents if len(sentence.text)>5]
    # print(split_sentences)
    sentences = merge_short_sentences(split_sentences)
    # print(sentences)
        
    fine_sentences = []
    for bold_index in range(len(sentences)):
        bold_sentence = f'<mark>{sentences[bold_index]}</mark>'
        new_sentences = sentences[:bold_index] + [bold_sentence] + sentences[bold_index + 1:]
        # print('new sentences!')
        # print(new_sentences)
        fine_sentence = ' '.join(new_sentences)
        clean_sentence = clean_fine_sentence(fine_sentence)
        fine_sentences.append((bold_index, clean_sentence))
    
    return fine_sentences

# Example text
text = """This is the first sentence. Here's the second one! Is this the third sentence? Yes, it is."""

# bold sentences in the example text
for i in bold_sentences(text):
    print(i)

(0, "<mark>This is the first sentence. Here's the second one!</mark> Is this the third sentence? Yes, it is.")
(1, "This is the first sentence. Here's the second one! <mark>Is this the third sentence? Yes, it is.</mark>")


## Putting pieces together

In [9]:
random.seed(17)

with open(os.path.join('data', 'fine_part_2.json'), 'r') as json_file:
    fine_questions = json.load(json_file)

fine_questions_batches = {}
for annotator, questions in fine_questions.items():
    fine_questions_batches[annotator] = {}
    batch_n = 1
    question_list = fine_questions[annotator].copy()
    random.shuffle(question_list)
    for i in range(0, len(question_list), 3):
            key = f'batch_{batch_n}'
            fine_questions_batches[annotator][key] = question_list[i:i+3]
            batch_n += 1

for annotator, batches in fine_questions_batches.items():
    print(annotator)
    total = 0
    for batch, questions in batches.items():
        print(len(questions))
        total += len(questions)
    print(total)

annotator1
3
3
3
3
3
3
3
2
23
annotator2
3
3
3
3
3
3
3
2
23
annotator3
3
3
3
3
3
3
3
2
23
annotator4
3
3
3
3
3
3
3
2
23
annotator5
3
3
3
3
3
3
3
2
23
annotator6
3
3
3
3
3
3
3
2
23


In [10]:
with open(os.path.join('data', 'new_wrong_ones.json'), 'r') as json_file:
    new_wrong_ones = json.load(json_file)

In [ ]:
# lengths = []
# for l in [gpt4_answers, llama_answers, physician_answers]:
#     for qa_pair in l:
#         if qa_pair['answer_id'] in wrong_ones.keys():
#             key = qa_pair['answer_id']
#             sentences = wrong_ones[key]
        
#         else:
#             sentences = bold_sentences(qa_pair['answer'])
            
#         d ={'answer_id': qa_pair['answer'],
#             'length': len(sentences)}
        
#         lengths.append(d)

# lengths_df = pd.DataFrame(lengths)


In [ ]:
# lengths_df['length'].describe()

count    300.000000
mean       5.716667
std        2.481313
min        1.000000
25%        4.000000
50%        5.000000
75%        7.000000
max       15.000000
Name: length, dtype: float64

In [11]:
questions_positions = {}
for n in range(len(gpt4_answers)):
    d = gpt4_answers[n]
    q_id = d['question_id']
    questions_positions[q_id] = [n, d['answer_id']]
questions_positions

{'question_78': [0, 'gpt4_0'],
 'question_24': [1, 'gpt4_1'],
 'question_6': [2, 'gpt4_2'],
 'question_156': [3, 'gpt4_3'],
 'question_82': [4, 'gpt4_4'],
 'question_112': [5, 'gpt4_5'],
 'question_101': [6, 'gpt4_6'],
 'question_139': [7, 'gpt4_7'],
 'question_95': [8, 'gpt4_8'],
 'question_36': [9, 'gpt4_9'],
 'question_190': [10, 'gpt4_10'],
 'question_52': [11, 'gpt4_11'],
 'question_71': [12, 'gpt4_12'],
 'question_117': [13, 'gpt4_13'],
 'question_4': [14, 'gpt4_14'],
 'question_59': [15, 'gpt4_15'],
 'question_163': [16, 'gpt4_16'],
 'question_120': [17, 'gpt4_17'],
 'question_8': [18, 'gpt4_18'],
 'question_30': [19, 'gpt4_19'],
 'question_199': [20, 'gpt4_20'],
 'question_131': [21, 'gpt4_21'],
 'question_111': [22, 'gpt4_22'],
 'question_7': [23, 'gpt4_23'],
 'question_135': [24, 'gpt4_24'],
 'question_165': [25, 'gpt4_25'],
 'question_114': [26, 'gpt4_26'],
 'question_192': [27, 'gpt4_27'],
 'question_16': [28, 'gpt4_28'],
 'question_55': [29, 'gpt4_29'],
 'question_11': [30

In [12]:
for annotator, batches in fine_questions_batches.items():
    print(annotator)
    for batch_id, questions in batches.items():
        print(batch_id)
        for question_id in questions:
            print(question_id)
            n  = questions_positions[question_id][0]
                
            qa_pairs = [gpt4_answers[n], llama_answers[n], physician_answers[n]] # Randomizing answer type order
            random.shuffle(qa_pairs)
            
            if qa_pairs[0]['question_id'] == qa_pairs[1]['question_id'] == qa_pairs[2]['question_id']:
            
                for qa_pair in qa_pairs:
                    
                    if qa_pair['answer_id'] in new_wrong_ones.keys():
                        key = qa_pair['answer_id']
                        sentences = new_wrong_ones[key]
                    
                    else:
                        sentences = bold_sentences(qa_pair['answer'])
                    
                    if len(sentences) >= 6:
                        sampled = random.sample(sentences, 6)
                    else:
                        sampled = sentences
                    
                    for sentence in sampled:
                        new_d = qa_pair.copy()
                        new_d['annotation_type'] = 'fine'
                        new_d['rated'] = 'No'
                        new_d['batch_id'] = batch_id
                        new_d['sentence_id'] = new_d['answer_id'] + f'_{sentence[0]}'
                        new_d['answer'] = sentence[1]
                        with open(os.path.join(output_dir, 'all', f'{annotator.lower()}_fine2_sampled.jsonl'), 'a') as file:
                            json.dump(new_d, file)
                            file.write('\n')

annotator1
batch_1
question_161
question_101
question_30
batch_2
question_118
question_47
question_52
batch_3
question_163
question_190
question_120
batch_4
question_76
question_148
question_16
batch_5
question_111
question_127
question_165
batch_6
question_199
question_98
question_78
batch_7
question_125
question_24
question_156
batch_8
question_117
question_44
annotator2
batch_1
question_55
question_4
question_199
batch_2
question_114
question_112
question_150
batch_3
question_16
question_98
question_117
batch_4
question_161
question_71
question_30
batch_5
question_111
question_24
question_49
batch_6
question_180
question_135
question_125
batch_7
question_8
question_96
question_120
batch_8
question_95
question_78
annotator3
batch_1
question_196
question_72
question_113
batch_2
question_74
question_20
question_33
batch_3
question_15
question_159
question_110
batch_4
question_94
question_23
question_73
batch_5
question_99
question_104
question_2
batch_6
question_83
question_191
questio

# Create pilot batches

In [16]:
l = ['llama_19_11', 'llama_19_3', 'llama_19_5', 'llama_19_7', 'llama_19_9']
dictionaries = []
for i in l:
    d = {'sentence_id': i}
    dictionaries.append(d)

In [20]:
dictionaries_dict = {'bibib': dictionaries}

In [21]:
sorted(dictionaries_dict['bibib'], key=lambda x: int(x['sentence_id'].split('_')[-1]))

[{'sentence_id': 'llama_19_3'},
 {'sentence_id': 'llama_19_5'},
 {'sentence_id': 'llama_19_7'},
 {'sentence_id': 'llama_19_9'},
 {'sentence_id': 'llama_19_11'}]

In [ ]:
question_ids_per_batch = 3

for annotation_type in ['coarse']:#,'fine']:
    
    with open(os.path.join(output_dir, 'pilot', f"pilot_{annotation_type}.jsonl"), 'r', encoding='utf-8') as jsonl_file:
        data = [json.loads(line) for line in jsonl_file]
    
    if annotation_type == 'coarse':
        data = data[9:18]
    else:
        data = data[:51]

    batch_ids = set()
    id_n = "X"
    batch_id = f'batch_{id_n}'

    for d in data:
        
        if d['question_id'] not in batch_ids and len(batch_ids) == question_ids_per_batch:
            batch_ids = set()
            id_n += 1
            batch_id = f'batch_{id_n}'
            
        d['batch_id'] = batch_id
    
    for d in data:
        with open(os.path.join(output_dir, 'pilot', f'pilot2_coarse.jsonl'), 'a') as file:
            json.dump(d, file)
            file.write('\n')

In [3]:
with open(os.path.join(output_dir, 'pilot', f"pilot1_fine_old.jsonl"), 'r', encoding='utf-8') as jsonl_file:
    data = [json.loads(line) for line in jsonl_file]
    
    for d in data:
        d['batch_id'] = 'batch_0'
        with open(os.path.join(output_dir, 'pilot', f'pilot1_fine.jsonl'), 'a') as file:
            json.dump(d, file)
            file.write('\n')

In [14]:
from collections import OrderedDict
import pymongo
from pymongo.mongo_client import MongoClient
import streamlit as st

uri = f"mongodb+srv://{open(os.path.join('..', '..', 'PhD', 'apikeys', 'mongodb_clinicalqa_uri.txt')).read().strip()}/?retryWrites=true&w=majority&appName=clinicalqa"
client = MongoClient(uri)  # Create a new client and connect to the server
db = client["fine2"]  # TEST DATABASE DO NOT CHANGE
annotator_n = '1'
batch_n = '1'

# annotator_n is the key used to access which set of annotations
# b/c we inserted based on annotator# in upload.py
annotations_collection = st.session_state.annotation_collection = db[
    f"annotator{annotator_n}"
]

mongodb_result = [
    i
    for i in annotations_collection.find(
        {"$and": [{"rated": "No"}, {"batch_id": f"batch_{batch_n}"}]}
    )
]  # check if any fine annotations left
print([d['answer_id'] for d in mongodb_result])
random.shuffle(mongodb_result)
print([d['answer_id'] for d in mongodb_result])

grouped = OrderedDict()
for item in mongodb_result:
    answer_id = item['answer_id']
    if answer_id not in grouped:
        grouped[answer_id] = []
    grouped[answer_id].append(item)
    
for aid in grouped:
    grouped[aid] = sorted(grouped[aid], key=lambda x: int(x['sentence_id'].split('_')[-1]))

clean_responses_todo = [item for group in grouped.values() for item in group]

for d in clean_responses_todo:
    print(d['sentence_id'])

2025-05-17 18:52:38.938 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-17 18:52:38.938 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


['llama_42', 'llama_42', 'llama_42', 'llama_42', 'gpt4_42', 'gpt4_42', 'gpt4_42', 'physician_42', 'physician_42', 'physician_42', 'physician_42', 'physician_6', 'physician_6', 'physician_6', 'physician_6', 'llama_6', 'llama_6', 'llama_6', 'llama_6', 'llama_6', 'llama_6', 'gpt4_6', 'gpt4_6', 'gpt4_6', 'gpt4_6', 'gpt4_19', 'gpt4_19', 'gpt4_19', 'gpt4_19', 'gpt4_19', 'gpt4_19', 'llama_19', 'llama_19', 'llama_19', 'llama_19', 'llama_19', 'llama_19', 'physician_19', 'physician_19', 'physician_19', 'physician_19', 'physician_19']
['physician_6', 'physician_6', 'gpt4_19', 'physician_42', 'gpt4_6', 'llama_19', 'gpt4_6', 'llama_6', 'gpt4_6', 'gpt4_42', 'llama_42', 'gpt4_6', 'llama_42', 'gpt4_19', 'gpt4_19', 'llama_19', 'physician_42', 'llama_6', 'physician_19', 'physician_42', 'gpt4_19', 'llama_6', 'llama_19', 'gpt4_42', 'llama_19', 'physician_19', 'gpt4_42', 'llama_19', 'gpt4_19', 'llama_6', 'physician_19', 'llama_19', 'llama_6', 'physician_42', 'physician_6', 'llama_6', 'gpt4_19', 'llama_42',

['llama_6', 'physician_19', 'gpt4_42', 'llama_6', 'llama_6', 'llama_42', 'llama_6', 'gpt4_19', 'llama_42', 'physician_19', 'gpt4_6', 'gpt4_42', 'llama_42', 'gpt4_6', 'llama_19', 'physician_19', 'gpt4_19', 'llama_6', 'gpt4_19', 'physician_19', 'llama_42', 'gpt4_19', 'physician_6', 'physician_19', 'physician_6', 'gpt4_6', 'gpt4_19', 'llama_19', 'physician_42', 'llama_6', 'gpt4_6', 'gpt4_19', 'physician_42', 'physician_6', 'llama_19', 'physician_42', 'llama_19', 'llama_19', 'physician_42', 'llama_19', 'gpt4_42', 'physician_6']
